In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
user = pd.read_csv('../input/训练数据-ccf_first_round_user_shop_behavior.csv')
shop = pd.read_csv('../input/训练数据-ccf_first_round_shop_info.csv')
test = pd.read_csv('../input/AB榜测试集-evaluation_public.csv')

user.shape
user.head()
shop.shape
shop.head()
test.shape
test.head()

IOError: File ../input/训练数据-ccf_first_round_user_shop_behavior.csv does not exist

In [15]:
user = user.merge(shop, left_on='shop_id', right_on='shop_id', how='left')
user['row_id'] = -1

del shop

user = user[['row_id', 'user_id', 'mall_id', 'wifi_infos', 'shop_id']]
test = test[['row_id', 'user_id', 'mall_id', 'wifi_infos']]

user.shape
user.head()
test.shape
test.head()

(1138015, 5)

,row_id,user_id,mall_id,wifi_infos,shop_id
0,-1,u_376,m_1409,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718
1,-1,u_376,m_1409,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718
2,-1,u_1041,m_4079,b_8006367|-78|false;b_2485110|-52|false;b_3005...,s_181637
3,-1,u_1158,m_6587,b_26250579|-73|false;b_26250580|-64|false;b_26...,s_609470
4,-1,u_1654,m_3005,b_39004150|-66|false;b_39004148|-58|false;b_21...,s_3816766


(483931, 4)

,row_id,user_id,mall_id,wifi_infos
0,118742,u_30097142,m_3916,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [16]:
test['shop_id'] = -1

full_data = pd.concat([user, test])

full_data.shape
full_data.head()

(1621946, 5)

,row_id,user_id,mall_id,wifi_infos,shop_id
0,-1,u_376,m_1409,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718
1,-1,u_376,m_1409,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718
2,-1,u_1041,m_4079,b_8006367|-78|false;b_2485110|-52|false;b_3005...,s_181637
3,-1,u_1158,m_6587,b_26250579|-73|false;b_26250580|-64|false;b_26...,s_609470
4,-1,u_1654,m_3005,b_39004150|-66|false;b_39004148|-58|false;b_21...,s_3816766


In [17]:
del user
del test

In [18]:
full_data.wifi_infos = full_data.wifi_infos.apply(lambda x : x.split(';'))
full_data['wifi_num'] = full_data.wifi_infos.apply(len)
full_data.head()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
0,-1,u_376,m_1409,"[b_6396480|-67|false, b_41124514|-86|false, b_...",s_2871718,10
1,-1,u_376,m_1409,"[b_6396480|-67|false, b_56328155|-73|false, b_...",s_2871718,10
2,-1,u_1041,m_4079,"[b_8006367|-78|false, b_2485110|-52|false, b_3...",s_181637,10
3,-1,u_1158,m_6587,"[b_26250579|-73|false, b_26250580|-64|false, b...",s_609470,10
4,-1,u_1654,m_3005,"[b_39004150|-66|false, b_39004148|-58|false, b...",s_3816766,10


In [19]:
len(full_data.mall_id.value_counts())
full_data.wifi_num.value_counts()

97

10    1492325
9       30890
8       25438
7       21666
6       17442
5       13367
4        9624
3        6368
2        3303
1        1511
20         10
18          1
16          1
Name: wifi_num, dtype: int64

In [20]:
def pop_wifi(x):
    if x != []:
        return x.pop()
    else:
        return x
    
for i in range(1, 11):
    full_data['wifi_info%d' % i] = full_data.wifi_infos.apply(pop_wifi)
    
full_data.head()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num,wifi_info1,wifi_info2,wifi_info3,wifi_info4,wifi_info5,wifi_info6,wifi_info7,wifi_info8,wifi_info9,wifi_info10
0,-1,u_376,m_1409,[],s_2871718,10,b_5857369|-55|false,b_56328155|-77|false,b_56326644|-89|false,b_5857370|-68|false,b_32053319|-74|false,b_8764723|-90|false,b_6396479|-55|false,b_28723327|-90|false,b_41124514|-86|false,b_6396480|-67|false
1,-1,u_376,m_1409,[],s_2871718,10,b_5857369|-57|false,b_31100514|-89|false,b_6396479|-57|false,b_5857370|-68|false,b_32053319|-83|false,b_56326319|-84|false,b_56328154|-73|false,b_41124514|-88|false,b_56328155|-73|false,b_6396480|-67|false
2,-1,u_1041,m_4079,[],s_181637,10,b_37608251|-84|false,b_35013153|-56|false,b_8006521|-74|false,b_27390312|-72|false,b_33503917|-63|false,b_33503892|-56|false,b_8006442|-71|false,b_30058118|-69|false,b_2485110|-52|false,b_8006367|-78|false
3,-1,u_1158,m_6587,[],s_609470,10,b_29510856|-80|false,b_26250578|-72|false,b_30424471|-60|false,b_26250517|-76|false,b_26250593|-81|false,b_52934996|-79|false,b_26250516|-77|false,b_26250581|-64|false,b_26250580|-64|false,b_26250579|-73|false
4,-1,u_1654,m_3005,[],s_3816766,10,b_21685901|-91|false,b_1845687|-72|false,b_6805211|-80|false,b_21685852|-81|false,b_21685851|-82|false,b_54466444|-86|false,b_1053646|-87|false,b_21685905|-90|false,b_39004148|-58|false,b_39004150|-66|false


In [21]:
full_data = full_data.drop(['wifi_infos'], axis=1)

def spl_wifi(x):
    if type(x) == str:
        return x.split('|')
    else:
        return x

for i in range(1, 11):
    full_data['wifi_info%d' % i] = full_data['wifi_info%d' % i].apply(spl_wifi)

In [155]:
full_data.head(10)

,row_id,user_id,mall_id,shop_id,wifi_num,wifi_info1,wifi_info2,wifi_info3,wifi_info4,wifi_info5,wifi_info6,wifi_info7,wifi_info8,wifi_info9,wifi_info10
0,-1,u_376,m_1409,s_2871718,10,"[b_5857369, -55, false]","[b_56328155, -77, false]","[b_56326644, -89, false]","[b_5857370, -68, false]","[b_32053319, -74, false]","[b_8764723, -90, false]","[b_6396479, -55, false]","[b_28723327, -90, false]","[b_41124514, -86, false]","[b_6396480, -67, false]"
1,-1,u_376,m_1409,s_2871718,10,"[b_5857369, -57, false]","[b_31100514, -89, false]","[b_6396479, -57, false]","[b_5857370, -68, false]","[b_32053319, -83, false]","[b_56326319, -84, false]","[b_56328154, -73, false]","[b_41124514, -88, false]","[b_56328155, -73, false]","[b_6396480, -67, false]"
2,-1,u_1041,m_4079,s_181637,10,"[b_37608251, -84, false]","[b_35013153, -56, false]","[b_8006521, -74, false]","[b_27390312, -72, false]","[b_33503917, -63, false]","[b_33503892, -56, false]","[b_8006442, -71, false]","[b_30058118, -69, false]","[b_2485110, -52, false]","[b_8006367, -78, false]"
3,-1,u_1158,m_6587,s_609470,10,"[b_29510856, -80, false]","[b_26250578, -72, false]","[b_30424471, -60, false]","[b_26250517, -76, false]","[b_26250593, -81, false]","[b_52934996, -79, false]","[b_26250516, -77, false]","[b_26250581, -64, false]","[b_26250580, -64, false]","[b_26250579, -73, false]"
4,-1,u_1654,m_3005,s_3816766,10,"[b_21685901, -91, false]","[b_1845687, -72, false]","[b_6805211, -80, false]","[b_21685852, -81, false]","[b_21685851, -82, false]","[b_54466444, -86, false]","[b_1053646, -87, false]","[b_21685905, -90, false]","[b_39004148, -58, false]","[b_39004150, -66, false]"
5,-1,u_2733,m_3839,s_3686420,10,"[b_19272603, -72, false]","[b_13784697, -71, false]","[b_21608835, -44, false]","[b_31160049, -59, false]","[b_12135674, -71, false]","[b_44402483, -71, false]","[b_2971974, -71, false]","[b_35861781, -78, false]","[b_7390858, -86, false]","[b_13784698, -75, false]"
6,-1,u_2848,m_1920,s_89874,10,"[b_32391921, -64, false]","[b_12363015, -82, false]","[b_7695545, -41, false]","[b_53066760, -70, false]","[b_975809, -37, false]","[b_12201872, -75, false]","[b_48005749, -46, false]","[b_35063048, -64, false]","[b_975807, -41, false]","[b_975810, -33, true]"
7,-1,u_3063,m_2878,s_939447,10,"[b_37749132, -67, false]","[b_37971796, -61, false]","[b_44597733, -61, false]","[b_25781993, -60, true]","[b_25781349, -68, false]","[b_13067253, -60, false]","[b_25674211, -75, false]","[b_41313117, -78, false]","[b_10120963, -76, false]","[b_15263246, -67, false]"
8,-1,u_3063,m_2878,s_939447,10,"[b_23648596, -63, false]","[b_18696482, -73, false]","[b_42938093, -34, false]","[b_25781993, -59, true]","[b_25781123, -70, false]","[b_25674211, -79, false]","[b_15263246, -68, false]","[b_25781979, -83, false]","[b_25781349, -63, false]","[b_37749132, -77, false]"
9,-1,u_3063,m_2878,s_3698335,10,"[b_47362601, -63, false]","[b_20903796, -64, false]","[b_15263246, -68, false]","[b_23354547, -64, false]","[b_23638651, -61, false]","[b_23353721, -61, false]","[b_25674236, -51, false]","[b_25674264, -59, false]","[b_25673942, -61, false]","[b_39846139, -73, false]"


In [82]:
def wifi_diff(x, y):
    name_y = []
    stre_y = []
    diff = 0
    for i in range(1, 11):
        if(y['wifi_info%d' % i] != []):
            name_y.append(y['wifi_info%d' % i][0])
            stre_y.append(int(y['wifi_info%d' % i][1]))
    for i in range(1, 11):
        if x['wifi_info%d' % i][0] in name_y:
            diff = diff + abs(stre_y[name_y.index(x['wifi_info%d' % i][0])] - int(x['wifi_info%d' % i][1]))
#             print(str(stre_y[name_y.index(x['wifi_info%d' % i][0])]) + ' ' + str(diff))
        else:
            diff = diff + abs(int(x['wifi_info%d' % i][1]))
#             print(diff)
    return diff

In [154]:
shop1_list = full_data[full_data.shop_id == 's_2871718'].index

a = []
for i in shop1_list[1:]:
    a.append(wifi_diff(full_data.iloc[0], full_data.iloc[i]))
    
a = pd.Series(a)
a.max()
a.min()
a.mean()

751

111

346.44262295081967

In [156]:
shop2_list = full_data[full_data.shop_id == 's_3698335'].index

b = []
for i in shop2_list:
    b.append(wifi_diff(full_data.iloc[8], full_data.iloc[i]))
    
b = pd.Series(b)
b.max()
b.min()
b.mean()

672

465

638.6516853932584

In [161]:
full_data.shape

(1621946, 15)

In [162]:
full_data.reset_index(drop=True, inplace=True)
full_data.tail()

,row_id,user_id,mall_id,shop_id,wifi_num,wifi_info1,wifi_info2,wifi_info3,wifi_info4,wifi_info5,wifi_info6,wifi_info7,wifi_info8,wifi_info9,wifi_info10
1621941,483927,u_63919729,m_7523,-1,10,"[b_30877522, -47, false]","[b_30876383, -68, false]","[b_53685771, -64, false]","[b_30877474, -69, false]","[b_37356762, -45, false]","[b_3331208, -75, false]","[b_30876622, -66, false]","[b_30897917, -54, true]","[b_8210864, -61, false]","[b_30877374, -71, false]"
1621942,483928,u_63920157,m_4828,-1,10,"[b_13274444, -62, false]","[b_9878209, -75, false]","[b_8032986, -64, false]","[b_56822214, -89, false]","[b_30141951, -82, false]","[b_8685803, -52, false]","[b_1075110, -62, false]","[b_16215145, -56, false]","[b_8685969, -74, false]","[b_8685813, -79, false]"
1621943,483929,u_63920157,m_4828,-1,8,"[b_26503035, -84, false]","[b_2207021, -69, false]","[b_39701808, -61, false]","[b_39837562, -70, false]","[b_29201055, -89, false]","[b_45089046, -52, true]","[b_34498163, -82, false]","[b_49160464, -65, false]",[],[]
1621944,483930,u_63920157,m_4828,-1,3,"[b_39701808, -59, false]","[b_49160464, -65, false]","[b_45089046, -47, true]",[],[],[],[],[],[],[]
1621945,483931,u_63920806,m_7973,-1,10,"[b_54838864, -51, false]","[b_39872253, -64, true]","[b_49255295, -80, false]","[b_55151564, -58, false]","[b_35654865, -80, false]","[b_50136089, -85, false]","[b_55151565, -74, false]","[b_25191669, -49, false]","[b_39872252, -57, false]","[b_25163301, -69, false]"


In [145]:
def shop_wifi_list(x):
    shop_wifi = []
    temp = full_data[full_data.shop_id == x].index
    for a in temp:
        for i in range(1, 11):
            if full_data.iloc[a]['wifi_info%d' % i] != []:
                shop_wifi.append(full_data.iloc[a]['wifi_info%d' % i][0])
    return set(shop_wifi)

def mall_wifi_list(x):
    mall_wifi = []
    temp = full_data[full_data.mall_id == x].index
    for a in temp:
        for i in range(1, 11):
            if full_data.iloc[a]['wifi_info%d' % i] != []:
                mall_wifi.append(full_data.iloc[a]['wifi_info%d' % i][0])
    return set(mall_wifi)

In [157]:
len(shop_wifi_list('s_2871718'))
len(mall_wifi_list('m_1409'))

69

38737

In [163]:
tmp = mall_wifi_list('m_1409')

In [164]:
len(set(tmp))

7092